# 2. Agenda-based Search

In [1]:
import pandas as pd

def load_data():
    """
    Loads data from tubedata.csv
    :return : data frame of the loaded data
    """
    #Reading data from tubedata.csv
    dataframe = pd.read_csv("tubedata.csv", header = None)
    return dataframe

In [2]:
dataframe = load_data()
dataframe.head()

,0,1,2,3,4,5
0,Harrow & Wealdstone,Kenton,Bakerloo,3,5,0
1,Kenton,South Kenton,Bakerloo,2,4,0
2,South Kenton,North Wembley,Bakerloo,2,4,0
3,North Wembley,Wembley Central,Bakerloo,2,4,0
4,Wembley Central,Stonebridge Park,Bakerloo,3,4,0


In [3]:
import networkx as nx

def create_station_graph(dataframe):
    """
    Creates a graph of the stations from the dataframe 
    : param dataframe : Pandas dataframe containing 
    : returns station_data : Graph of the stations created from the dataframe passed to it
    """
    station_data = nx.Graph()
    for index, row in dataframe.iterrows():
        start_station = row[0]
        end_station = row[1]
        tube_line = row[2]
        time = float(row[3])
        main_zone = row[4]
        sec_zone = row[5]
        
        station_data.add_edge(start_station, end_station, time = time, tube_line = tube_line, main_zone= main_zone, sec_zone= sec_zone)
 
    return station_data

In [4]:
station_data = create_station_graph(dataframe)

# 2.1 Implement BFS, DFS, and UCS

In [5]:
class Station(): 
    """
    Used to represent the stations for BFS, DFS and UCS
    """
    def __init__(self, name, parent = None, time = None, tube_line = None):
        """
        Initializes a station with the station details passed to it
        : param name : Name of the current station
        : param parent : The name of the previously expanded station
        : param time : Average time in minutes taken between the current station and parent station
        : param tube_line : The line that connects current station and the parent station
        """
        self.name = name
        self.parent = parent
        self.time = time
        self.tube_line = tube_line


In [6]:
def cost_of_path(end_station):
    """
    Finds the path that was taken by backtracking from the end station to the start station
    :param end_station : The end station that has to be reached
    :returns : The path that was taken from start station to the end station and the time taken to reach end station from start station
    """
    path = []
    time = 0
    station = end_station
    
    #Start station is reached when the station does not have a parent
    while station.parent is not None: 
        path.append(station.name)
        time += (station.time)
        station = station.parent

    path.append(station.name)
    
    #Reversing path as path contains stations from end station to start station
    path.reverse()
    
    return path, time



# DFS

In [7]:
def dfs(station_data, start_station_name, end_station_name, reverse = False):
    """
    Implements depth first search algorithm 
    :returns path: The path found by the depth first search algorithm from start to end station
    :returns time_taken : The time taken to travel from start to end station by the dfs
    :num_of_explored_stations : The number of stations that was expanded by dfs
    """
    start_station = Station(start_station_name)
    
    frontier = []
         
    frontier.append(start_station)
    
    explored_stations = set()
    explored_stations.add(start_station_name)
    
    num_of_explored_stations = 0     
    
    #Looking for the best path until frontier is not empty
    while frontier:
        current_station= frontier.pop()
        num_of_explored_stations += 1
            
            
        if current_station.name == end_station_name:
            path, time_taken = cost_of_path(current_station)
            return path, time_taken, num_of_explored_stations            
            
        neighbours = reversed(list(station_data.neighbors(current_station.name))) if reverse else station_data.neighbors(current_station.name)
            
        for neighbour in neighbours:
            child_station_time = station_data[current_station.name][neighbour]["time"]
            child_station_tube = station_data[current_station.name][neighbour]["tube_line"]
            
            child_station = Station(neighbour, current_station, child_station_time, child_station_tube)
            
            #Child station added to the frontier and explored stations
            if neighbour not in explored_stations:                                      
                frontier.append(child_station)
                explored_stations.add(child_station.name)

In [8]:
path, time_taken, num_of_stations_explored = dfs(station_data, "Euston", "Victoria")
print("Path found by Depth first search algorithm from Euston to Victoria is : \n" + str(path))
print("Time taken to travel from Euston to Victoria : ", time_taken)
print("The number of nodes explored on the path from Euston to Victoria is : ", num_of_stations_explored)


Path found by Depth first search algorithm from Euston to Victoria is : 
['Euston', "King's Cross St. Pancras", 'Russell Square', 'Holborn', 'Covent Garden', 'Leicester Square', 'Piccadilly Circus', 'Green Park', 'Victoria']
Time taken to travel from Euston to Victoria :  13.0
The number of nodes explored on the path from Euston to Victoria is :  25


# BFS

In [9]:
import collections

def bfs(station_data, start_station_name, end_station_name, reverse = False):  
    """
    Implements bredth first search algorithm 
    :returns path: The path found by the bredth first search algorithm from start to end station
    :returns time_taken : The time taken to travel from start to end station by the bfs
    :num_of_explored_stations : The number of stations that was expanded by bfs
    """
    start_station = Station(start_station_name)
    
    frontier = collections.deque()
         
    frontier.append(start_station)
    
    explored_stations = set()
    explored_stations.add(start_station_name)
    num_of_explored_stations = 0     
    
    #Looking for the best path until frontier is not empty
    while frontier: 
        current_station= frontier.popleft()
        num_of_explored_stations += 1
            
            
        if current_station.name == end_station_name:
            path, time_taken = cost_of_path(current_station)
            return path, time_taken, num_of_explored_stations
        
        neighbours = reversed(list(station_data.neighbors(current_station.name))) if reverse else station_data.neighbors(current_station.name)
            
        for neighbour in neighbours:
            child_station_time = station_data[current_station.name][neighbour]["time"]
            child_station_tube = station_data[current_station.name][neighbour]["tube_line"]
                   
            child_station = Station(neighbour, current_station, child_station_time, child_station_tube)

            #Child station added to the frontier and explored stations
            if neighbour not in explored_stations:                                      
                frontier.append(child_station)
                explored_stations.add(child_station.name)
                

In [10]:
path, time_taken, num_of_stations_explored = bfs(station_data, "Euston", "Victoria")
print("Path found by Bredth First Search algorithm from Euston to Victoria is : \n" + str(path))
print("Time taken to travel from Euston to Victoria : ", time_taken)
print("The number of nodes explored on the path from Euston to Victoria is : ", num_of_stations_explored)


Path found by Bredth First Search algorithm from Euston to Victoria is : 
['Euston', 'Warren Street', 'Oxford Circus', 'Green Park', 'Victoria']
Time taken to travel from Euston to Victoria :  7.0
The number of nodes explored on the path from Euston to Victoria is :  35


# UCS

In [11]:
class Ucs():

    def __init__(self):
        self.frontier=[]
        #The stations added to the frontier and the time taken to reach that station added as a frontier
        self.frontier_details= []
        
    def add_station(self,station): 
        if station.time == None: 
            self.frontier.append(station)
            self.frontier_details.append([station, 0])
        else:
            self.frontier.append(station)
            
            time_taken = 0 
            child_station = station 
            
            #The sum of time taken from starting station to current station in calculated and stored in time_taken
            while child_station.time is not None: 
                time_taken += child_station.time  
                child_station = child_station.parent
            
            self.frontier_details.append([station, time_taken])   
            
            #Sorting the stations wrt to time taken to reach the stations
            frontier_details_sorted= sorted(self.frontier_details, key=lambda x:x[1], reverse=False)
            
            frontier_sorted = []
            for f in frontier_details_sorted:
                if f[0] in self.frontier:
                    frontier_sorted.append(f[0])
                    
            self.frontier = frontier_sorted.copy()
    
    def remove_station(self): 
        if len(self.frontier) == 0: 
            raise Exception ("The frontier is empty")
        else:                  
            station=self.frontier[0]
            self.frontier=self.frontier[1:]
            return station
           

In [12]:
def ucs(station_data, start_station_name, end_station_name, reverse = False):
    
    start_station = Station(start_station_name)
    
    frontier = Ucs()
         
    frontier.add_station(start_station)
    
    explored_stations = set()
    explored_stations.add(start_station_name)
    num_of_explored_stations = 0     
    
    #Looking for the best path until frontier is not empty
    while frontier: 
        
        current_station= frontier.remove_station()
        num_of_explored_stations += 1
            
            
        if current_station.name == end_station_name:
            path, time_taken = cost_of_path(current_station)
            return path, time_taken, num_of_explored_stations
            
        neighbours = reversed(list(station_data.neighbors(current_station.name))) if reverse else station_data.neighbors(current_station.name)
            
        for neighbour in neighbours:
            child_station_time = station_data[current_station.name][neighbour]["time"]
            child_station_tube = station_data[current_station.name][neighbour]["tube_line"]
                
                
            child_station = Station(neighbour, current_station, child_station_time, child_station_tube)

            #Child station added to the frontier and explored stations
            if neighbour not in explored_stations:                                      
                frontier.add_station(child_station)
                explored_stations.add(child_station.name)

In [13]:
path, time_taken, num_of_stations_explored = ucs(station_data, "Euston", "Victoria")
print("Path found by Uniform Cost Search algorithm from Euston to Victoria is : \n" + str(path))
print("Time taken to travel from Euston to Victoria : ", time_taken)
print("The number of nodes explored on the path from Euston to Victoria is : ", num_of_stations_explored)


Path found by Uniform Cost Search algorithm from Euston to Victoria is : 
['Euston', 'Warren Street', 'Oxford Circus', 'Green Park', 'Victoria']
Time taken to travel from Euston to Victoria :  7.0
The number of nodes explored on the path from Euston to Victoria is :  30


# 2.2 Compare DFS, BFS, and USC

### Direct order of explored nodes at each level 

In [14]:
start_station = "Canada Water"
end_station = "Stratford"

path, time_taken, num_of_stations_explored = dfs(station_data, start_station, end_station)
print("Path found by Depth First Search algorithm from " + start_station + " to " + end_station + " is : \n" + str(path))
print("Time taken to travel from " + start_station + " to " + end_station + " : " , time_taken)
print("The number of nodes explored on the path from " + start_station + " to " + end_station + " is : ", num_of_stations_explored)
print("\n")

path, time_taken, num_of_stations_explored = bfs(station_data, start_station, end_station)
print("Path found by Bredth First Search algorithm from " + start_station + " to " + end_station + " is : \n" + str(path))
print("Time taken to travel from " + start_station + " to " + end_station + " : " , time_taken)
print("The number of nodes explored on the path from " + start_station + " to " + end_station + " is : ", num_of_stations_explored)
print("\n")

path, time_taken, num_of_stations_explored = ucs(station_data, start_station, end_station)
print("Path found by Uniform Cost Search algorithm from " + start_station + " to " + end_station + " is : \n" + str(path))
print("Time taken to travel from " + start_station + " to " + end_station + " : " , time_taken)
print("The number of nodes explored on the path from " + start_station + " to " + end_station + " is : ", num_of_stations_explored)


Path found by Depth First Search algorithm from Canada Water to Stratford is : 
['Canada Water', 'Canary Wharf', 'North Greenwich', 'Canning Town', 'West Ham', 'Stratford']
Time taken to travel from Canada Water to Stratford :  15.0
The number of nodes explored on the path from Canada Water to Stratford is :  6


Path found by Bredth First Search algorithm from Canada Water to Stratford is : 
['Canada Water', 'Canary Wharf', 'North Greenwich', 'Canning Town', 'West Ham', 'Stratford']
Time taken to travel from Canada Water to Stratford :  15.0
The number of nodes explored on the path from Canada Water to Stratford is :  40


Path found by Uniform Cost Search algorithm from Canada Water to Stratford is : 
['Canada Water', 'Rotherhithe', 'Wapping', 'Shadwell', 'Whitechapel', 'Stepney Green', 'Mile End', 'Stratford']
Time taken to travel from Canada Water to Stratford :  14.0
The number of nodes explored on the path from Canada Water to Stratford is :  52


In [15]:
start_station = "New Cross Gate"
end_station = "Stepney Green"

path, time_taken, num_of_stations_explored = dfs(station_data, start_station, end_station)
print("Path found by Depth First Search algorithm from " + start_station + " to " + end_station + " is : \n" + str(path))
print("Time taken to travel from " + start_station + " to " + end_station + " : " , time_taken)
print("The number of nodes explored on the path from " + start_station + " to " + end_station + " is : ", num_of_stations_explored)
print("\n")

path, time_taken, num_of_stations_explored = bfs(station_data, start_station, end_station)
print("Path found by Bredth First Search algorithm from " + start_station + " to " + end_station + " is : \n" + str(path))
print("Time taken to travel from " + start_station + " to " + end_station + " : " , time_taken)
print("The number of nodes explored on the path from " + start_station + " to " + end_station + " is : ", num_of_stations_explored)
print("\n")

path, time_taken, num_of_stations_explored = ucs(station_data, start_station, end_station)
print("Path found by Uniform Cost Search algorithm from " + start_station + " to " + end_station + " is : \n" + str(path))
print("Time taken to travel from " + start_station + " to " + end_station + " : " , time_taken)
print("The number of nodes explored on the path from " + start_station + " to " + end_station + " is : ", num_of_stations_explored)


Path found by Depth First Search algorithm from New Cross Gate to Stepney Green is : 
['New Cross Gate', 'Surrey Quays', 'Canada Water', 'Canary Wharf', 'North Greenwich', 'Canning Town', 'West Ham', 'Stratford', 'Mile End', 'Stepney Green']
Time taken to travel from New Cross Gate to Stepney Green :  27.0
The number of nodes explored on the path from New Cross Gate to Stepney Green is :  32


Path found by Bredth First Search algorithm from New Cross Gate to Stepney Green is : 
['New Cross Gate', 'Surrey Quays', 'Canada Water', 'Rotherhithe', 'Wapping', 'Shadwell', 'Whitechapel', 'Stepney Green']
Time taken to travel from New Cross Gate to Stepney Green :  14.0
The number of nodes explored on the path from New Cross Gate to Stepney Green is :  26


Path found by Uniform Cost Search algorithm from New Cross Gate to Stepney Green is : 
['New Cross Gate', 'Surrey Quays', 'Canada Water', 'Rotherhithe', 'Wapping', 'Shadwell', 'Whitechapel', 'Stepney Green']
Time taken to travel from New Cr

In [16]:
start_station = "Ealing Broadway"
end_station = "South Kensington"

path, time_taken, num_of_stations_explored = dfs(station_data, start_station, end_station)
print("Path found by Depth First Search algorithm from " + start_station + " to " + end_station + " is : \n" + str(path))
print("Time taken to travel from " + start_station + " to " + end_station + " : " , time_taken)
print("The number of nodes explored on the path from " + start_station + " to " + end_station + " is : ", num_of_stations_explored)
print("\n")

path, time_taken, num_of_stations_explored = bfs(station_data, start_station, end_station)
print("Path found by Bredth First Search algorithm from " + start_station + " to " + end_station + " is : \n" + str(path))
print("Time taken to travel from " + start_station + " to " + end_station + " : " , time_taken)
print("The number of nodes explored on the path from " + start_station + " to " + end_station + " is : ", num_of_stations_explored)
print("\n")

path, time_taken, num_of_stations_explored = ucs(station_data, start_station, end_station)
print("Path found by Uniform Cost Search algorithm from " + start_station + " to " + end_station + " is : \n" + str(path))
print("Time taken to travel from " + start_station + " to " + end_station + " : " , time_taken)
print("The number of nodes explored on the path from " + start_station + " to " + end_station + " is : ", num_of_stations_explored)


Path found by Depth First Search algorithm from Ealing Broadway to South Kensington is : 
['Ealing Broadway', 'Ealing Common', 'North Ealing', 'Park Royal', 'Alperton', 'Sudbury Town', 'Sudbury Hill', 'South Harrow', 'Rayners Lane', 'West Harrow', 'Harrow-on-the-Hill', 'Northwick Park', 'Preston Road', 'Wembley Park', 'Finchley Road', 'Baker Street', 'Bond Street', 'Green Park', 'Victoria', 'Sloane Square', 'South Kensington']
Time taken to travel from Ealing Broadway to South Kensington :  57.0
The number of nodes explored on the path from Ealing Broadway to South Kensington is :  179


Path found by Bredth First Search algorithm from Ealing Broadway to South Kensington is : 
['Ealing Broadway', 'Ealing Common', 'Acton Town', 'Turnham Green', 'Hammersmith', 'Barons Court', "Earls' Court", 'Gloucester Road', 'South Kensington']
Time taken to travel from Ealing Broadway to South Kensington :  20.0
The number of nodes explored on the path from Ealing Broadway to South Kensington is :  50

In [17]:
start_station = "Baker Street"
end_station = "Wembley Park"

path, time_taken, num_of_stations_explored = dfs(station_data, start_station, end_station)
print("Path found by Depth First Search algorithm from " + start_station + " to " + end_station + " is : \n" + str(path))
print("Time taken to travel from " + start_station + " to " + end_station + " : " , time_taken)
print("The number of nodes explored on the path from " + start_station + " to " + end_station + " is : ", num_of_stations_explored)
print("\n")

path, time_taken, num_of_stations_explored = bfs(station_data, start_station, end_station)
print("Path found by Bredth First Search algorithm from " + start_station + " to " + end_station + " is : \n" + str(path))
print("Time taken to travel from " + start_station + " to " + end_station + " : " , time_taken)
print("The number of nodes explored on the path from " + start_station + " to " + end_station + " is : ", num_of_stations_explored)
print("\n")

path, time_taken, num_of_stations_explored = ucs(station_data, start_station, end_station)
print("Path found by Uniform Cost Search algorithm from " + start_station + " to " + end_station + " is : \n" + str(path))
print("Time taken to travel from " + start_station + " to " + end_station + " : " , time_taken)
print("The number of nodes explored on the path from " + start_station + " to " + end_station + " is : ", num_of_stations_explored)


Path found by Depth First Search algorithm from Baker Street to Wembley Park is : 
['Baker Street', 'Finchley Road', 'Wembley Park']
Time taken to travel from Baker Street to Wembley Park :  13.0
The number of nodes explored on the path from Baker Street to Wembley Park is :  3


Path found by Bredth First Search algorithm from Baker Street to Wembley Park is : 
['Baker Street', 'Finchley Road', 'Wembley Park']
Time taken to travel from Baker Street to Wembley Park :  13.0
The number of nodes explored on the path from Baker Street to Wembley Park is :  16


Path found by Uniform Cost Search algorithm from Baker Street to Wembley Park is : 
['Baker Street', 'Finchley Road', 'Wembley Park']
Time taken to travel from Baker Street to Wembley Park :  13.0
The number of nodes explored on the path from Baker Street to Wembley Park is :  70


### Reverse order of explored nodes at each level

In [18]:
start_station = "Canada Water"
end_station = "Stratford"

path, time_taken, num_of_stations_explored = dfs(station_data, start_station, end_station, True)
print("Path found by Depth First Search algorithm from " + start_station + " to " + end_station + " is : \n" + str(path))
print("Time taken to travel from " + start_station + " to " + end_station + " : " , time_taken)
print("The number of nodes explored on the path from " + start_station + " to " + end_station + " is : ", num_of_stations_explored)
print("\n")

path, time_taken, num_of_stations_explored = bfs(station_data, start_station, end_station, True)
print("Path found by Bredth First Search algorithm from " + start_station + " to " + end_station + " is : \n" + str(path))
print("Time taken to travel from " + start_station + " to " + end_station + " : " , time_taken)
print("The number of nodes explored on the path from " + start_station + " to " + end_station + " is : ", num_of_stations_explored)
print("\n")

path, time_taken, num_of_stations_explored = ucs(station_data, start_station, end_station, True)
print("Path found by Uniform Cost Search algorithm from " + start_station + " to " + end_station + " is : \n" + str(path))
print("Time taken to travel from " + start_station + " to " + end_station + " : " , time_taken)
print("The number of nodes explored on the path from " + start_station + " to " + end_station + " is : ", num_of_stations_explored)


Path found by Depth First Search algorithm from Canada Water to Stratford is : 
['Canada Water', 'Rotherhithe', 'Wapping', 'Shadwell', 'Whitechapel', 'Aldgate East', 'Liverpool Street', 'Bethnal Green', 'Mile End', 'Stratford']
Time taken to travel from Canada Water to Stratford :  20.0
The number of nodes explored on the path from Canada Water to Stratford is :  227


Path found by Bredth First Search algorithm from Canada Water to Stratford is : 
['Canada Water', 'Canary Wharf', 'North Greenwich', 'Canning Town', 'West Ham', 'Stratford']
Time taken to travel from Canada Water to Stratford :  15.0
The number of nodes explored on the path from Canada Water to Stratford is :  25


Path found by Uniform Cost Search algorithm from Canada Water to Stratford is : 
['Canada Water', 'Rotherhithe', 'Wapping', 'Shadwell', 'Whitechapel', 'Stepney Green', 'Mile End', 'Stratford']
Time taken to travel from Canada Water to Stratford :  14.0
The number of nodes explored on the path from Canada Water

In [19]:
start_station = "New Cross Gate"
end_station = "Stepney Green"

path, time_taken, num_of_stations_explored = dfs(station_data, start_station, end_station, True)
print("Path found by Depth First Search algorithm from " + start_station + " to " + end_station + " is : \n" + str(path))
print("Time taken to travel from " + start_station + " to " + end_station + " : " , time_taken)
print("The number of nodes explored on the path from " + start_station + " to " + end_station + " is : ", num_of_stations_explored)
print("\n")

path, time_taken, num_of_stations_explored = bfs(station_data, start_station, end_station, True)
print("Path found by Bredth First Search algorithm from " + start_station + " to " + end_station + " is : \n" + str(path))
print("Time taken to travel from " + start_station + " to " + end_station + " : " , time_taken)
print("The number of nodes explored on the path from " + start_station + " to " + end_station + " is : ", num_of_stations_explored)
print("\n")

path, time_taken, num_of_stations_explored = ucs(station_data, start_station, end_station, True)
print("Path found by Uniform Cost Search algorithm from " + start_station + " to " + end_station + " is : \n" + str(path))
print("Time taken to travel from " + start_station + " to " + end_station + " : " , time_taken)
print("The number of nodes explored on the path from " + start_station + " to " + end_station + " is : ", num_of_stations_explored)


Path found by Depth First Search algorithm from New Cross Gate to Stepney Green is : 
['New Cross Gate', 'Surrey Quays', 'Canada Water', 'Rotherhithe', 'Wapping', 'Shadwell', 'Whitechapel', 'Stepney Green']
Time taken to travel from New Cross Gate to Stepney Green :  14.0
The number of nodes explored on the path from New Cross Gate to Stepney Green is :  267


Path found by Bredth First Search algorithm from New Cross Gate to Stepney Green is : 
['New Cross Gate', 'Surrey Quays', 'Canada Water', 'Rotherhithe', 'Wapping', 'Shadwell', 'Whitechapel', 'Stepney Green']
Time taken to travel from New Cross Gate to Stepney Green :  14.0
The number of nodes explored on the path from New Cross Gate to Stepney Green is :  40


Path found by Uniform Cost Search algorithm from New Cross Gate to Stepney Green is : 
['New Cross Gate', 'Surrey Quays', 'Canada Water', 'Rotherhithe', 'Wapping', 'Shadwell', 'Whitechapel', 'Stepney Green']
Time taken to travel from New Cross Gate to Stepney Green :  14.0


In [20]:
start_station = "Ealing Broadway"
end_station = "South Kensington"

path, time_taken, num_of_stations_explored = dfs(station_data, start_station, end_station, True)
print("Path found by Depth First Search algorithm from " + start_station + " to " + end_station + " is : \n" + str(path))
print("Time taken to travel from " + start_station + " to " + end_station + " : " , time_taken)
print("The number of nodes explored on the path from " + start_station + " to " + end_station + " is : ", num_of_stations_explored)
print("\n")

path, time_taken, num_of_stations_explored = bfs(station_data, start_station, end_station, True)
print("Path found by Bredth First Search algorithm from " + start_station + " to " + end_station + " is : \n" + str(path))
print("Time taken to travel from " + start_station + " to " + end_station + " : " , time_taken)
print("The number of nodes explored on the path from " + start_station + " to " + end_station + " is : ", num_of_stations_explored)
print("\n")

path, time_taken, num_of_stations_explored = ucs(station_data, start_station, end_station, True)
print("Path found by Uniform Cost Search algorithm from " + start_station + " to " + end_station + " is : \n" + str(path))
print("Time taken to travel from " + start_station + " to " + end_station + " : " , time_taken)
print("The number of nodes explored on the path from " + start_station + " to " + end_station + " is : ", num_of_stations_explored)


Path found by Depth First Search algorithm from Ealing Broadway to South Kensington is : 
['Ealing Broadway', 'West Acton', 'North Acton', 'East Acton', 'White City', "Shepherd's Bush", 'Holland Park', 'Notting Hill Gate', 'Queensway', 'Lancaster Gate', 'Marble Arch', 'Bond Street', 'Oxford Circus', 'Piccadilly Circus', 'Charing Cross', 'Embankment', 'Waterloo', 'Kennington', 'Oval', 'Stockwell', 'Vauxhall', 'Pimlico', 'Victoria', 'Sloane Square', 'South Kensington']
Time taken to travel from Ealing Broadway to South Kensington :  51.0
The number of nodes explored on the path from Ealing Broadway to South Kensington is :  157


Path found by Bredth First Search algorithm from Ealing Broadway to South Kensington is : 
['Ealing Broadway', 'Ealing Common', 'Acton Town', 'Turnham Green', 'Hammersmith', 'Barons Court', "Earls' Court", 'Gloucester Road', 'South Kensington']
Time taken to travel from Ealing Broadway to South Kensington :  20.0
The number of nodes explored on the path from Eal

In [21]:
start_station = "Baker Street"
end_station = "Wembley Park"

path, time_taken, num_of_stations_explored = dfs(station_data, start_station, end_station, True)
print("Path found by Depth First Search algorithm from " + start_station + " to " + end_station + " is : \n" + str(path))
print("Time taken to travel from " + start_station + " to " + end_station + " : " , time_taken)
print("The number of nodes explored on the path from " + start_station + " to " + end_station + " is : ", num_of_stations_explored)
print("\n")

path, time_taken, num_of_stations_explored = bfs(station_data, start_station, end_station, True)
print("Path found by Bredth First Search algorithm from " + start_station + " to " + end_station + " is : \n" + str(path))
print("Time taken to travel from " + start_station + " to " + end_station + " : " , time_taken)
print("The number of nodes explored on the path from " + start_station + " to " + end_station + " is : ", num_of_stations_explored)
print("\n")

path, time_taken, num_of_stations_explored = ucs(station_data, start_station, end_station, True)
print("Path found by Uniform Cost Search algorithm from " + start_station + " to " + end_station + " is : \n" + str(path))
print("Time taken to travel from " + start_station + " to " + end_station + " : " , time_taken)
print("The number of nodes explored on the path from " + start_station + " to " + end_station + " is : ", num_of_stations_explored)


Path found by Depth First Search algorithm from Baker Street to Wembley Park is : 
['Baker Street', "Regent's Park", 'Oxford Circus', 'Piccadilly Circus', 'Charing Cross', 'Embankment', 'Waterloo', 'Kennington', 'Oval', 'Stockwell', 'Vauxhall', 'Pimlico', 'Victoria', 'Sloane Square', 'South Kensington', 'Gloucester Road', 'High Street Kensington', 'Notting Hill Gate', 'Holland Park', "Shepherd's Bush", 'White City', 'East Acton', 'North Acton', 'West Acton', 'Ealing Broadway', 'Ealing Common', 'North Ealing', 'Park Royal', 'Alperton', 'Sudbury Town', 'Sudbury Hill', 'South Harrow', 'Rayners Lane', 'West Harrow', 'Harrow-on-the-Hill', 'Northwick Park', 'Preston Road', 'Wembley Park']
Time taken to travel from Baker Street to Wembley Park :  89.0
The number of nodes explored on the path from Baker Street to Wembley Park is :  191


Path found by Bredth First Search algorithm from Baker Street to Wembley Park is : 
['Baker Street', 'Finchley Road', 'Wembley Park']
Time taken to travel fro

# 2.3 Extending Cost Function

In [22]:
class UcsImproved():

    def __init__(self):
        self.frontier = []
        #The stations added to the frontier and the time taken to reach that station added as a frontier
        self.frontier_details = []
        
    def add_station(self,station): 
        #Startng station is added to the frontier with time taken to reach the station as 0
        if station.time == None: 
            self.frontier.append(station)
            self.frontier_details.append([station, 0])
        else:
            self.frontier.append(station)
            
            time_taken =0 
            child_station = station 
            
            while child_station.time is not None: 
                time_taken += child_station.time 
                child_station = child_station.parent
             
            tube_line_parent=station.parent.tube_line
            tube_line_child = station.tube_line
            if tube_line_parent is not None:
                 
                if tube_line_parent != tube_line_child:
                    time_taken += 2 
            
            self.frontier_details.append([station, time_taken])
            
            frontier_details_sorted= sorted(self.frontier_details, key=lambda x:x[1], reverse=False)
            
            frontier_sorted = []
            for x in frontier_details_sorted:
                if x[0] in self.frontier:
                    frontier_sorted.append(x[0])
                    
            self.frontier = frontier_sorted.copy()
            
    def remove_station(self): 
        if len(self.frontier) == 0: 
            raise Exception ("The frontier is empty")
        else: 
                        
            station=self.frontier[0]
            self.frontier=self.frontier[1:]
            return station

In [23]:
def ucs_improved(station_data, start_station_name, end_station_name, reverse = False):
    start_station = Station(start_station_name)
     
    frontier = UcsImproved()
   
    frontier.add_station(start_station)
    
    explored_stations=set()
    explored_stations.add((start_station_name, None))
    
    num_of_explored_stations = 0
    
    #Looking for the best path until frontier is not empty
    while frontier: 
        current_station = frontier.remove_station()

        num_of_explored_stations += 1
            
        if current_station.name == end_station_name:
            path, time_taken = cost_of_path(current_station)
            return path, time_taken, num_of_explored_stations
            
        neighbours = reversed(list(station_data.neighbors(current_station.name))) if reverse else station_data.neighbors(current_station.name)
            
        for neighbour in neighbours:
            child_station_time =station_data[current_station.name][neighbour]["time"]
            child_station_tube_line =station_data[current_station.name][neighbour]["tube_line"]
  
            child_station = Station(neighbour, current_station, child_station_time, child_station_tube_line)

            #Child station added to the frontier and explored stations
            if (neighbour, child_station_tube_line) not in explored_stations:                                      
                frontier.add_station(child_station)
                explored_stations.add((child_station.name, child_station.tube_line))           


In [24]:
path, time_taken, num_of_stations_explored = ucs_improved(station_data, "Euston", "Victoria")
print("Path found by Uniform Cost Search algorithm from Euston to Victoria is : \n" + str(path))
print("Time taken to travel from Euston to Victoria : ", time_taken)
print("The number of nodes explored on the path from Euston to Victoria is : ", num_of_stations_explored)


Path found by Uniform Cost Search algorithm from Euston to Victoria is : 
['Euston', 'Warren Street', 'Oxford Circus', 'Green Park', 'Victoria']
Time taken to travel from Euston to Victoria :  7.0
The number of nodes explored on the path from Euston to Victoria is :  30


Canada Water to Stratford

In [25]:
start_station = "Canada Water"
end_station = "Stratford"

path, time_taken, num_of_stations_explored = ucs(station_data, start_station, end_station)
print("Path found by Uniform Cost Search algorithm from " + start_station + " to " + end_station + " is : \n" + str(path))
print("Time taken to travel from " + start_station + " to " + end_station + " : " , time_taken)
print("The number of nodes explored on the path from " + start_station + " to " + end_station + " is : ", num_of_stations_explored)
print("\n")

path, time_taken, num_of_stations_explored = ucs_improved(station_data, start_station, end_station)
print("Path found by Improved Uniform Cost Search algorithm from " + start_station + " to " + end_station + " is : \n" + str(path))
print("Time taken to travel from " + start_station + " to " + end_station + " : " , time_taken)
print("The number of nodes explored on the path from " + start_station + " to " + end_station + " is : ", num_of_stations_explored)


Path found by Uniform Cost Search algorithm from Canada Water to Stratford is : 
['Canada Water', 'Rotherhithe', 'Wapping', 'Shadwell', 'Whitechapel', 'Stepney Green', 'Mile End', 'Stratford']
Time taken to travel from Canada Water to Stratford :  14.0
The number of nodes explored on the path from Canada Water to Stratford is :  52


Path found by Improved Uniform Cost Search algorithm from Canada Water to Stratford is : 
['Canada Water', 'Canary Wharf', 'North Greenwich', 'Canning Town', 'West Ham', 'Stratford']
Time taken to travel from Canada Water to Stratford :  15.0
The number of nodes explored on the path from Canada Water to Stratford is :  74


New Cross Gate to Stepney Green

In [26]:
start_station = "New Cross Gate"
end_station = "Stepney Green"

path, time_taken, num_of_stations_explored = ucs(station_data, start_station, end_station)
print("Path found by Uniform Cost Search algorithm from " + start_station + " to " + end_station + " is : \n" + str(path))
print("Time taken to travel from " + start_station + " to " + end_station + " : " , time_taken)
print("The number of nodes explored on the path from " + start_station + " to " + end_station + " is : ", num_of_stations_explored)
print("\n")

path, time_taken, num_of_stations_explored = ucs_improved(station_data, start_station, end_station)
print("Path found by Improved Uniform Cost Search algorithm from " + start_station + " to " + end_station + " is : \n" + str(path))
print("Time taken to travel from " + start_station + " to " + end_station + " : " , time_taken)
print("The number of nodes explored on the path from " + start_station + " to " + end_station + " is : ", num_of_stations_explored)


Path found by Uniform Cost Search algorithm from New Cross Gate to Stepney Green is : 
['New Cross Gate', 'Surrey Quays', 'Canada Water', 'Rotherhithe', 'Wapping', 'Shadwell', 'Whitechapel', 'Stepney Green']
Time taken to travel from New Cross Gate to Stepney Green :  14.0
The number of nodes explored on the path from New Cross Gate to Stepney Green is :  18


Path found by Improved Uniform Cost Search algorithm from New Cross Gate to Stepney Green is : 
['New Cross Gate', 'Surrey Quays', 'Canada Water', 'Rotherhithe', 'Wapping', 'Shadwell', 'Whitechapel', 'Stepney Green']
Time taken to travel from New Cross Gate to Stepney Green :  14.0
The number of nodes explored on the path from New Cross Gate to Stepney Green is :  25


Ealing Broadway to South Kensington

In [27]:
start_station = "Ealing Broadway"
end_station = "South Kensington"

path, time_taken, num_of_stations_explored = ucs(station_data, start_station, end_station)
print("Path found by Uniform Cost Search algorithm from " + start_station + " to " + end_station + " is : \n" + str(path))
print("Time taken to travel from " + start_station + " to " + end_station + " : " , time_taken)
print("The number of nodes explored on the path from " + start_station + " to " + end_station + " is : ", num_of_stations_explored)
print("\n")

path, time_taken, num_of_stations_explored = ucs_improved(station_data, start_station, end_station)
print("Path found by Improved Uniform Cost Search algorithm from " + start_station + " to " + end_station + " is : \n" + str(path))
print("Time taken to travel from " + start_station + " to " + end_station + " : " , time_taken)
print("The number of nodes explored on the path from " + start_station + " to " + end_station + " is : ", num_of_stations_explored)


Path found by Uniform Cost Search algorithm from Ealing Broadway to South Kensington is : 
['Ealing Broadway', 'Ealing Common', 'Acton Town', 'Turnham Green', 'Hammersmith', 'Barons Court', "Earls' Court", 'Gloucester Road', 'South Kensington']
Time taken to travel from Ealing Broadway to South Kensington :  20.0
The number of nodes explored on the path from Ealing Broadway to South Kensington is :  53


Path found by Improved Uniform Cost Search algorithm from Ealing Broadway to South Kensington is : 
['Ealing Broadway', 'Ealing Common', 'Acton Town', 'Turnham Green', 'Hammersmith', 'Barons Court', "Earls' Court", 'Gloucester Road', 'South Kensington']
Time taken to travel from Ealing Broadway to South Kensington :  20.0
The number of nodes explored on the path from Ealing Broadway to South Kensington is :  58


Baker Street to Wembley Park

In [28]:
start_station = "Baker Street"
end_station = "Wembley Park"

path, time_taken, num_of_stations_explored = ucs(station_data, start_station, end_station)
print("Path found by Uniform Cost Search algorithm from " + start_station + " to " + end_station + " is : \n" + str(path))
print("Time taken to travel from " + start_station + " to " + end_station + " : " , time_taken)
print("The number of nodes explored on the path from " + start_station + " to " + end_station + " is : ", num_of_stations_explored)
print("\n")

path, time_taken, num_of_stations_explored = ucs_improved(station_data, start_station, end_station)
print("Path found by Improved Uniform Cost Search algorithm from " + start_station + " to " + end_station + " is : \n" + str(path))
print("Time taken to travel from " + start_station + " to " + end_station + " : " , time_taken)
print("The number of nodes explored on the path from " + start_station + " to " + end_station + " is : ", num_of_stations_explored)


Path found by Uniform Cost Search algorithm from Baker Street to Wembley Park is : 
['Baker Street', 'Finchley Road', 'Wembley Park']
Time taken to travel from Baker Street to Wembley Park :  13.0
The number of nodes explored on the path from Baker Street to Wembley Park is :  70


Path found by Improved Uniform Cost Search algorithm from Baker Street to Wembley Park is : 
['Baker Street', 'Finchley Road', 'Wembley Park']
Time taken to travel from Baker Street to Wembley Park :  13.0
The number of nodes explored on the path from Baker Street to Wembley Park is :  89


# Heuristic Search

In [29]:
class StationDetails(): 
    """
    Used to represent the stations for BFS, DFS and UCS
    """
    def __init__(self, name, parent = None, time = None, tube_line = None, main_zone = None, sec_zone = None):
        """
        Initializes a station with the station details passed to it
        : param name : Name of the current station
        : param parent : The name of the previously expanded station
        : param time : Average time in minutes taken between the current station and parent station
        : param tube_line : The line that connects current station and the parent station
        : param main_zone : Main zone of the current station
        : param sec_zone : Secondary zone of the current station
        """
        self.name = name
        self.parent = parent
        self.time = time
        self.tube_line = tube_line
        self.main_zone = main_zone
        self.sec_zone = sec_zone

In [30]:
from collections import defaultdict

def create_zone_dictionary(dataframe):
    zone_dict = defaultdict(set)
    for index, row in dataframe.iterrows():
        start_station = row[0]
        end_station = row[1]
        main_zone = row[4]
        sec_zone = row[5]
        # we add the main zone
        zone_dict[start_station].add(main_zone) 
    
        # we add the secondary zone
        if sec_zone != "0":
            zone_dict[start_station].add(sec_zone)
        
            # if the secondary zone is not 0 it’s the main zone for the ending station 
            zone_dict[end_station].add(sec_zone)
    
        else:
            # otherwise the main zone for the ending station is the same as the starting station 
            zone_dict[end_station].add(main_zone)
    
    zone_dictionary = defaultdict(list)
    for station in zone_dict:
        zone_list = []
        for zone in zone_dict[station]:
            zone_list.append(zone)
        if len(zone_list) == 1:
            zone_list.append(zone_list[0])
        zone_dictionary[station] = zone_list
    return zone_dictionary
        
        

In [31]:
zone_dictionary = create_zone_dictionary(dataframe)

In [32]:
class BestFirstSearch():

    def __init__(self):
        self.frontier=[]
        self.frontier_details= []
        
    def add_station(self,station, end_main_zone, end_sec_zone, current_main_zone, current_sec_zone): 
        
        if station.time == None: 
            self.frontier.append(station)
            self.frontier_details.append([station, 0])
        else:
            self.frontier.append(station)
           
            heursitic = 0 
            child_station = station   
            
            tube_line_parent= station.parent.tube_line
            tube_line_child = station.tube_line
                          
            parent = station.parent
            parent_main_zone, parent_sec_zone = zone_dictionary[parent.name]
            
            zone_number = {"1": 1, "2": 2, "3": 3, "4":4, "5": 5, "6": 6, "a": 7, "b": 8, "c": 9, "d": 10}  
            end_main_zone = zone_number[end_main_zone]
            end_sec_zone = zone_number[end_sec_zone]
            
            current_main_zone = zone_number[current_main_zone]
            current_sec_zone = zone_number[current_sec_zone]
            
            parent_main_zone = zone_number[parent_main_zone]
            parent_sec_zone = zone_number[parent_sec_zone]
            
            #If the end station and the current station is the same zone then heuristic value is not changed
            if not (current_main_zone == end_main_zone or current_sec_zone == end_main_zone or current_main_zone == end_sec_zone or current_sec_zone == end_sec_zone):
                
                #If the previous station and the current station are in the same zone then heuristics increased by 3
                if parent_main_zone == current_main_zone or parent_main_zone == current_sec_zone or parent_sec_zone == current_main_zone or parent_sec_zone == current_sec_zone:
                    heursitic += 3
                    
                #If we are moving in the corrct direction higher zone to lower zone then heuristics increased by 2
                elif current_main_zone >end_main_zone or current_sec_zone > end_main_zone or current_main_zone >end_sec_zone or current_sec_zone>end_sec_zone: 
                    if parent_main_zone == current_main_zone + 1 or parent_main_zone == current_sec_zone + 1 or parent_sec_zone == current_main_zone + 1 or parent_sec_zone == current_sec_zone+1:
                        heursitic += 2
                    else:
                        #If not moving in the correct direction heuristics increased by 4
                        heursitic += 4
                else:
                    #If we are moving in the corrct direction lower zone to higher zone then heuristics increased by 2
                    if parent_main_zone == current_main_zone - 1 or parent_main_zone == current_sec_zone-1 or parent_sec_zone == current_main_zone - 1 or parent_sec_zone == current_sec_zone - 1:
                        heursitic += 2
                    else:
                        #If not moving in the correct direction heuristics increased by 4
                        heursitic += 4

            

            self.frontier_details.append([station, heursitic])
            
            frontier_details_sorted = sorted(self.frontier_details, key=lambda x:x[1], reverse=False)
            
            frontier_sorted = []
            for f in frontier_details_sorted:
                if f[0] in self.frontier:
                    frontier_sorted.append(f[0])         
            
            self.frontier = frontier_sorted.copy()  
    
    def remove_station(self): 
        if len(self.frontier) == 0: 
            raise Exception ("The frontier is empty")
        else:                
            station=self.frontier[0]
            self.frontier = self.frontier[1:]
            return station        
            


In [33]:
def best_first_search(station_data, start_station_name, end_station_name, zone_dictionary):
    
    start_main_zone, start_sec_zone = zone_dictionary[start_station_name]
    
    start_station = StationDetails(start_station_name)
    
     
    frontier = BestFirstSearch()
    
    
    frontier.add_station(start_station, None, None, None, None)
    
    explored_stations = set()
    
    explored_stations.add((start_station_name, None))
    
    num_of_explored_stations = 0 
    
    end_main_zone, end_sec_zone = zone_dictionary[end_station_name]
        
    
    #Looking for the best path until frontier is not empty
    while frontier: 
        
        current_station= frontier.remove_station()  
        num_of_explored_stations += 1
                        
            
        if current_station.name == end_station_name:
            route, time_taken = cost_of_path(current_station)
            return route, time_taken, num_of_explored_stations        
            
        neighbours = station_data.neighbors(current_station.name) 
            
        for neighbour in neighbours:
            child_station_time = station_data[current_station.name][neighbour]["time"]
            child_station_tube = station_data[current_station.name][neighbour]["tube_line"]
            
            child_main_zone, child_secondary_zone = zone_dictionary[neighbour]
            
            child_station = StationDetails(neighbour, current_station, child_station_time, child_station_tube, child_main_zone, child_secondary_zone)
                
            #Child station added to the frontier and explored stations   
            if (neighbour, child_station_tube) not in explored_stations:                                      
                frontier.add_station(child_station, end_main_zone, end_sec_zone, child_main_zone, child_secondary_zone)
                explored_stations.add((child_station.name, child_station.tube_line))
                            
 

In [34]:
path, time_taken, num_of_stations_explored = best_first_search(station_data, "Euston", "Victoria", zone_dictionary)
print("Path found by Best First Search algorithm from Euston to Victoria is : \n" + str(path))
print("Time taken to travel from Euston to Victoria : ", time_taken)
print("The number of nodes explored on the path from Euston to Victoria is : ", num_of_stations_explored)


Path found by Best First Search algorithm from Euston to Victoria is : 
['Euston', 'Warren Street', 'Oxford Circus', 'Green Park', 'Victoria']
Time taken to travel from Euston to Victoria :  7.0
The number of nodes explored on the path from Euston to Victoria is :  40


In [35]:
start_station = "Canada Water"
end_station = "Stratford"

path, time_taken, num_of_stations_explored = best_first_search(station_data, start_station, end_station, zone_dictionary)
print("Path found by Best First Search algorithm from " + start_station + " to " + end_station + " is : \n" + str(path))
print("Time taken to travel from " + start_station + " to " + end_station + " : " , time_taken)
print("The number of nodes explored on the path from " + start_station + " to " + end_station + " is : ", num_of_stations_explored)


Path found by Best First Search algorithm from Canada Water to Stratford is : 
['Canada Water', 'Canary Wharf', 'North Greenwich', 'Canning Town', 'West Ham', 'Stratford']
Time taken to travel from Canada Water to Stratford :  15.0
The number of nodes explored on the path from Canada Water to Stratford is :  20


In [36]:
start_station = "New Cross Gate"
end_station = "Stepney Green"

path, time_taken, num_of_stations_explored = best_first_search(station_data, start_station, end_station, zone_dictionary)
print("Path found by Best First Search algorithm from " + start_station + " to " + end_station + " is : \n" + str(path))
print("Time taken to travel from " + start_station + " to " + end_station + " : " , time_taken)
print("The number of nodes explored on the path from " + start_station + " to " + end_station + " is : ", num_of_stations_explored)


Path found by Best First Search algorithm from New Cross Gate to Stepney Green is : 
['New Cross Gate', 'Surrey Quays', 'Canada Water', 'Rotherhithe', 'Wapping', 'Shadwell', 'Whitechapel', 'Stepney Green']
Time taken to travel from New Cross Gate to Stepney Green :  14.0
The number of nodes explored on the path from New Cross Gate to Stepney Green is :  14


In [37]:
start_station = "Ealing Broadway"
end_station = "South Kensington"

path, time_taken, num_of_stations_explored = best_first_search(station_data, start_station, end_station, zone_dictionary)
print("Path found by Best First Search algorithm from " + start_station + " to " + end_station + " is : \n" + str(path))
print("Time taken to travel from " + start_station + " to " + end_station + " : " , time_taken)
print("The number of nodes explored on the path from " + start_station + " to " + end_station + " is : ", num_of_stations_explored)


Path found by Best First Search algorithm from Ealing Broadway to South Kensington is : 
['Ealing Broadway', 'Ealing Common', 'Acton Town', 'Turnham Green', 'Hammersmith', 'Barons Court', "Earls' Court", 'Gloucester Road', 'South Kensington']
Time taken to travel from Ealing Broadway to South Kensington :  20.0
The number of nodes explored on the path from Ealing Broadway to South Kensington is :  36


In [38]:
start_station = "Baker Street"
end_station = "Wembley Park"

path, time_taken, num_of_stations_explored = best_first_search(station_data, start_station, end_station, zone_dictionary)
print("Path found by Best First Search algorithm from " + start_station + " to " + end_station + " is : \n" + str(path))
print("Time taken to travel from " + start_station + " to " + end_station + " : " , time_taken)
print("The number of nodes explored on the path from " + start_station + " to " + end_station + " is : ", num_of_stations_explored)


Path found by Best First Search algorithm from Baker Street to Wembley Park is : 
['Baker Street', 'Finchley Road', 'Wembley Park']
Time taken to travel from Baker Street to Wembley Park :  13.0
The number of nodes explored on the path from Baker Street to Wembley Park is :  8


# 3. Genetic Algorithm

In [39]:
from hints import closeness
import random

In [40]:
class Chromosome:
    """
    Used to represent the secret phrase and its fitness
    """
    def __init__(self, secret_phrase, fitness = None):
        """
        Initializes with the secret phrase and the fitness
        :param secret_phrase : A string of the gussed phrase
        :param fitness : Fitness of the secret phrase obtained from closeness function
        """
        self.secret_phrase = secret_phrase
        self.fitness = fitness
 

In [41]:
def compute_fitness(population):
    """
    Computes the fitness of the list of Chromosomes passed to it
    :param population : List containing the chromosomes
    :returns population: List of chromosomes containg the secret phrase and their fitness value obtained from closeness function
    """
    population_list = []
    for chromosome in population:
        population_list.append(chromosome.secret_phrase)
        
    fitness_dictionary = closeness(population_list, USERNAME)
    
    population = []
    for phrase in fitness_dictionary:
        population.append(Chromosome(phrase, fitness_dictionary[phrase]))
    return population

In [42]:
def phrase_generation():
    """
    Generate random strings having length as the PHRASE_LENGTH
    :returns string : Randomly generated phrase
    """
    genes = list("")
    for i in range(PHRASE_LENGTH):
        genes.append(random.choice(GENE_SET))
    return ''.join(genes)

In [43]:
def initialise_population():
    """
    Initialises the population with a list of chromosomes having size of list as the given population size and fitness value as None
    :returns population : List of chromosomes with fitness value as None
    """
    population = []
    for i in range(POPULATION_SIZE):
        population.append(Chromosome(phrase_generation()))
    return population

In [44]:
def selection(population):
    """
    Selects two chromosomes having highest fitness value as parent from a random pool of TOURNAMENT_SIZE
    :param population : List of chromosomes containing secret phrase and fitness value
    :returns parent_1, parent_2 : Returns two chromosomes having higest fitness value
    """
    tournament_list = []
    
    for i in range(TOURNAMENT_SIZE):
        tournament_list.append(random.choice(population))
    
    tournament_list = sorted(tournament_list, key = lambda chromosome: chromosome.fitness, reverse = True)
    
    parent_1 = tournament_list[0]
    parent_2 = tournament_list[1]
    
    return parent_1, parent_2

In [45]:
def crossover(parent_1, parent_2):
    """
    Crossover is done between two fittest parents to produce two children at random crossover points
    :param parent_1 : Parent 1
    :param parent_2 : Parent 2
    :returns : Two Chromosomes containg secret phrase obtained by crossover and fitness value as None
    """
    phrase_1 = parent_1.secret_phrase
    phrase_2 = parent_2.secret_phrase
    
    for i in range(CROSSOVER_POINTS):
        crossover_point = random.randint(0, PHRASE_LENGTH - 1)
        
        crossover_phrase_1 = phrase_1[ : crossover_point] + phrase_2[crossover_point : ]
        crossover_phrase_2 = phrase_2[ : crossover_point] + phrase_1[crossover_point : ]
        
        phrase_1 = crossover_phrase_1
        phrase_2 = crossover_phrase_2
    
    return Chromosome(phrase_1), Chromosome(phrase_2)
        

In [46]:
def mutation(population):
    """
    Perform mutation on the list of Chromosomes by mutating a random gene with a probability
    :param population : List of chromosomes
    :return population : List of chromosomes with mutated gene
    """
    for individual in population:
        if random.uniform(0, 1) < MUTATION_PROBABILITY:
            position = random.randint(0, PHRASE_LENGTH - 1)
            
            mutated_gene = random.choice(GENE_SET)
            
            phrase = list(individual.secret_phrase)
            
            phrase[position] = mutated_gene
            
            individual.secret_phrase = "".join(phrase)
    return population

In [48]:
#random.seed(10)

#List of hyperparameters
POPULATION_SIZE = 100
MUTATION_PROBABILITY = 0.04
TOURNAMENT_SIZE = 4
CROSSOVER_POINTS = 3

PHRASE_LENGTH = 10
GENE_SET = "ABCDEFGHIJKLMNOPQRSTUVWXYZ_0123456789" 

#Username(Student Number)
USERNAME = "username"

count_of_generation = 1

#Initialising the first generation with random strings having fitness as None
initial_population = initialise_population()

#Computing the fitness of the first generation of population
population = compute_fitness(initial_population)

#Sorting the first generation population wrt to fitness in decreasing order
sorted_population = sorted(population, key = lambda chromosome : chromosome.fitness, reverse = True)

#Looping until the secret phrase is found(fitness of secret phrase = 1)
while sorted_population[0].fitness != 1.0:
    next_generation = list()
    
    for i in range(int(POPULATION_SIZE / 2)):
        #Selecting two parents from the population
        parent_1, parent_2 = selection(population)
        
        #Crossover being done on the selected parents
        child_1, child_2 = crossover(parent_1, parent_2)
        
        #Adding the childrent produced to the next generation
        next_generation.append(child_1)
        next_generation.append(child_2)
    
    #Mutation performed on the next generation
    population = mutation(next_generation)
    
    
    population = compute_fitness(population)
    
    sorted_population = sorted(population, key = lambda chromosome : chromosome.fitness, reverse = True)
    
    count_of_generation += 1
    
    
secret_phrase = sorted_population[0].secret_phrase
print("Secret Phrase : ", secret_phrase)
print("Number of reproductions needed to converge to secret phrase : ", count_of_generation)
        

Secret Phrase :  CAJYS7_ID7
Number of reproductions needed to converge to secret phrase :  183
